## Импорт и определение библиотек

In [24]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [25]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q
!pip install sentence-transformers numpy transformers requests -q

In [ ]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
#from langchain.tools.retriever import create_retriever_tool
#from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
import tqdm as notebook_tqdm
from huggingface_hub import HfApi

In [4]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [ ]:
# Чтение токена из файла
def read_token(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": token
                  } 
)

## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [ ]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

In [ ]:
# для gpt-3.5-turbo
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],  # Разделители для сохранения структуры
    chunk_size=1000,  # 1000 символов (~200-250 токенов)
    chunk_overlap=200  # 200 символов перекрытия
)

# для gpt-4o-mini
"""splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=2000,  # 2000 символов (~400-500 токенов)
    chunk_overlap=300  # 300 символов перекрытия
)"""

# Разделяем книгу на части
#documents = splitter.split_documents(doc_epub)

split_documents = []
for page in doc_epub:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

In [ ]:
chunks = splitter.split_documents(doc_epub)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

### Наполним чашу `Vector Store`

In [12]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [28]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [ ]:
"""query = "грейпфрутовый сок, ржаной виски"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)"""

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [29]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем API ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            course_api_key = line.split('=')[1].strip()
            break

# Инициализация языковой модели
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",         #"gpt-4o-mini", 
    temperature=0.2, 
    course_api_key=course_api_key
)

# объявление retriever 
retriever = db_embed.as_retriever(
        search_type="mmr",              # тип поиска похожих документов
        search_kwargs={
                "k": 5,                 # количество возвращаемых документов (Default: 4)
                "fetch_k": 20
            },     
    )


retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm, 
        retriever=retriever,
        return_source_documents=True    # Возвращает исходные документы
    )

# Месье системный промт

In [30]:
# Системный промт для агента

system_prompt = """
        Ты — помощник для поиска рецептов коктейлей. У тебя есть доступ к базе данных с рецептами, вдохновлёнными литературными произведениями.
        Когда пользователь вводит ингредиенты, используй инструмент "Cocktail Recipe Finder", чтобы найти подходящие рецепты.
        Если рецепт не найден, предложи альтернативные варианты или уточни запрос.

        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].
    """

# Миксуем рецепт коктейля `agent` 00х

In [ ]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [32]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "по-умолчанию")


In [ ]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "по-умолчанию"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt).content
        
        # Переформулировка заголовков
        styled_response = styled_response.replace("Коктейль:", "Название коктейля:") 
        styled_response = styled_response.replace("Ингредиенты:", "Состав:") 
        styled_response = styled_response.replace("Приготовление:", "Способ приготовления:")
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()
    # Теперь переменная recipe содержит рецепт коктейля
    #print("Сохраненный рецепт:", recipe)

Добро пожаловать в мир коктейлей, где таинственные ингредиенты сочетаются в непостижимых пропорциях, призывая древние силы из глубин космоса и хтонических миров! 

1. Коктейль "Ржаной сурфер":
- 60 мл ржаного виски;
- 90 мл грейпфрутового сока.

Способ приготовления: смешайте эти загадочные ингредиенты в шейкере с ледяным дыханием, взболтайте и пролейте в бокал. Украсьте ломтиком грейпфрута, чтобы привлечь внимание древних богов.

2. Коктейль "Грейпфрутовый виски-сорбет":
- 45 мл ржаного виски;
- 60 мл грейпфрутового сока.

Способ приготовления: объедините эти таинственные ингредиенты в блендере с льдом, пока не получится однородная масса. Наполните стакан этим зельем и подайте с трубочкой, чтобы пробудить древние страхи и ужасы.

Погрузитесь в мир загадочных коктейлей, который открывает двери в мир Лавкрафтовского ужаса и мрака!
Сохраненный рецепт: Добро пожаловать в мир коктейлей, где таинственные ингредиенты сочетаются в непостижимых пропорциях, призывая древние силы из глубин космо

## Теперь озвучка

In [89]:
import requests
import uuid

def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None

In [90]:
# Считываем ключ авторизации из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('SBER_API_KEY'):
            SBER_API_KEY = line.split('=', 1)[1].strip(' \n\r')
            if not SBER_API_KEY.endswith('=='):  # Если нет ==, добавляем их
                SBER_API_KEY += '=='
            break

print(f"Прочитанное значение: '{SBER_API_KEY}'")
#'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='

response = get_token(SBER_API_KEY)
if response != None:
    salute_token = response.json()['access_token']

Прочитанное значение: 'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='


In [92]:
# синтез речи
def synthesize_speech(text, token, format='wav16', voice='May_24000'):
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/text"
    }
    params = {
        "format": format,
        "voice": voice
    }
    response = requests.post(url, headers=headers, params=params, data=text.encode(), verify=False)

    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open('output.wav', 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")

In [102]:
synthesize_speech(recipe, salute_token)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'


In [ ]:
"""import requests
import json

# Считываем API ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('SBER_API_KEY'):
            SBER_API_KEY = line.split('=')[1].strip()
            break

# Замените на ваши данные
#API_URL = 'https://api.sbercloud.ru/salute/v1/speech/synthesize'
API_URL = "https://smartspeech.sber.ru/rest/v1/text:synthesizeе"

def synthesize_speech(text):
    headers = {
        'Authorization': f'Bearer {SBER_API_KEY}',
        'Content-Type': 'application/json',
        'Accept': 'audio/mpeg'  # Ожидаем получить аудиофайл в формате MP3
    }
    
    data = {
        'text': text,
        'voice': 'May_24000',  # Вы можете выбрать другой голос, если необходимо
        'emotion': 'neutral',
        'speed': 1.0
    }
    
    try:
        response = requests.post(API_URL, headers=headers, data=json.dumps(data), verify=False)
        response.raise_for_status()  # Выбросит исключение для статусов 4xx и 5xx
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP ошибка: {http_err}")
        print(f"Ответ сервера: {response.text}")
    except Exception as err:
        print(f"Произошла ошибка: {err}")
    else:
        with open('output.mp3', 'wb') as audio_file:
            audio_file.write(response.content)
        print("Аудиофайл успешно создан.")"""

In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      #repo_id="IlyaGusev/saiga_llama3_8b",
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = course_api_key
    )"""